In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [52]:
!ls "/content/drive/My Drive/capstone"

 combined_csv_forecast.csv   train_label.p   wiki-news-300d-1M.vec
'Q&A_Database_new.xlsx'      val_input.p
 train_input.p		     val_label.p


In [0]:
!pip install -q keras

In [5]:
!pip3 install sacrebleu

  Running setup.py bdist_wheel for sacrebleu ... - done
  Stored in directory: /root/.cache/pip/wheels/ea/0a/7d/ddcbdcd15a04b72de1b3f78e7e754aab415aff81c423376385
Successfully built sacrebleu


In [0]:
import numpy as np
from numpy import array
import pickle as plk
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Bidirectional
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
import os
from sacrebleu import sentence_bleu

In [9]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 6.1MB/s 


In [0]:
path = os.getcwd()+ '/drive/My Drive/capstone'
df = pd.read_excel(path+'/Q&A_Database_new.xlsx','QA', skiprows=3)
df2 = pd.read_csv(path+ '/combined_csv_forecast.csv', encoding='latin-1')

In [0]:
df['Breakout'] = [x.strip() for x in df['Breakout'].values]
l = df['Breakout'].unique() # 14 unique categories
dic = {} #create dictionary for questions
for category in l:
  list_1 = list(df.loc[df['Breakout'] == category]['Question'])
  dic[category] = list_1
l2 = df2['sub_lvl'].unique()
dic_trans = {}
for x in l2:
    sub = x.split('-')[1]
    if sub != 'CIB' and sub!= 'CB' and sub!='CCB' and sub!='AWM':
        dic_trans[x] = sub[0]+sub.lower()[1:]
    else:
        dic_trans[x] = sub
#create dictionary for questions
for category in l2:
    cat = dic_trans[category]
    list_ = list(df2.loc[df2['sub_lvl']  == category]['Question'])
    dic[cat]+=list_

In [0]:
import spacy
import re
import string
# Load English tokenizer, tagger, parser, NER and word vectors

# tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

#tokenize sentence by sentence
def question_split(input_):
    list_ = []
    for q in input_:
        q = q.split('\n')
        if len(q) ==1 and len(tokenize(q[0]))>= 10:
            list_.append(q)
        else:
            for i in range(len(q)):
                if len(tokenize(q[i])) >= 10:
                    list_.append(q[i])
    return list_

def tokenize(sent):
#   sent = re.sub('[^A-Za-z&]', ' ', sent) # replace non-letter with space
#   sent = re.sub(r'\b[a-zA-Z]\b', '', sent) #remove single letter 
    sent = re.sub('y ou', 'you', sent)
    sent = re.sub('y es', 'yes', sent)
    sent = re.sub('v o', 'vo', sent)
    sent = re.sub("don't", 'dont', sent)
    sent = re.sub('[^A-Za-z&]', ' ', sent)
    sent = re.sub( r'([a-zA-Z])([,.!?])', r'\1 \2', sent)
#     tokens = tokenizer(sent)
    return sent.split()



In [0]:
#load training and validation dataset
CATEGORY = 'Balance sheet'
dic_len = len(dic[CATEGORY])
all_data = dic[CATEGORY]
split_val = int(dic_len*0.8)
train_data =dic[CATEGORY][:split_val]
val_data = dic[CATEGORY][split_val: ]

In [0]:
# organize into sequences of tokens
def build_sequence(length, input_):
    input_ = ['<s>'] + input_ + ['</s>']
    if len(input_) < length:
        input_ = input_ + ['<pad>'] * (length - len(input_))
    sequences = list()
    for l in range(length, len(input_)+1):
        seq = input_[l-length:l]
        line = ' '.join(seq)
        sequences.append(line)
#     print('Total Sequences: {}' .format(len(sequences)))
    return sequences

In [0]:
def data_generator(dic, length):
    out_list = []
    raw = question_split(dic)
    for i in range(len(raw)):
        sent = tokenize(raw[i][0].lower())
        if len(sent) >= 10:
            q_sequence = build_sequence(length, sent)
            out_list += q_sequence
    return out_list

In [0]:
all_data_words = []
for i in range(len(all_data)):
  sent = tokenize(all_data[i].lower())
  all_data_words += sent
all_data_words = list(set(all_data_words))

In [0]:
# add words not in the embeddings
words_to_load = 50000
PAD_IDX = 0
UNK_IDX = 1
SOS_IDX = 2
EOS_IDX = 3
import numpy as np
# reserve the 1st 2nd token for padding and <UNK> respectively
with open(path+'/wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft_en = np.zeros((words_to_load+4, 300))
    words_ft_en = {}
    idx2words_ft_en = {}
    ordered_words_ft_en = []
    ordered_words_ft_en.extend(['<pad>', '<unk>', '<s>', '</s>'])
    loaded_embeddings_ft_en[0,:] = np.zeros(300)
    loaded_embeddings_ft_en[1,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[2,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[3,:] = np.random.normal(size = 300)
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft_en[i+4, :] = np.asarray(s[1:])
        words_ft_en[s[0]] = i+4
        idx2words_ft_en[i+4] = s[0]
        ordered_words_ft_en.append(s[0])
    length = len(np.setdiff1d(all_data_words, ordered_words_ft_en))
    tmp_embeddings = np.zeros((length, 300))
    for idx, word in enumerate(np.setdiff1d(all_data_words, ordered_words_ft_en)):
        words_ft_en[word] = idx+words_to_load+4
        idx2words_ft_en[idx+words_to_load+4] = word
        tmp_embeddings[idx, :] = np.random.normal(size = 300)
    loaded_embeddings_ft_en = np.concatenate((loaded_embeddings_ft_en, tmp_embeddings), axis = 0)
    words_ft_en['<pad>'] = PAD_IDX
    words_ft_en['<unk>'] = UNK_IDX
    words_ft_en['<s>'] = SOS_IDX
    words_ft_en['</s>'] = EOS_IDX
    idx2words_ft_en[PAD_IDX] = '<pad>'
    idx2words_ft_en[UNK_IDX] = '<unk>'
    idx2words_ft_en[SOS_IDX] = '<s>'
    idx2words_ft_en[EOS_IDX] = '</s>'
    ordered_words_ft_en = list(words_ft_en.keys())

In [0]:
import nltk
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 30

In [0]:
all_d = data_generator(all_data,MAX_SEQUENCE_LENGTH)
train_d = data_generator(train_data,MAX_SEQUENCE_LENGTH)
val_d = data_generator(val_data,MAX_SEQUENCE_LENGTH)

In [0]:
from keras.optimizers import RMSprop
def data_loader(data_input):
    X = []
    y = []
    for i in range(len(data_input)):
        X.append([words_ft_en[x] if x in ordered_words_ft_en else UNK_IDX for x in data_input[i].split()[:-1]])
        sub = data_input[i].split()[-1]
        if sub in ordered_words_ft_en:
            y.append(loaded_embeddings_ft_en[words_ft_en[sub]])
        else:
            y.append(loaded_embeddings_ft_en[UNK_IDX])
    return np.array(X), np.array(y)
# train_input, train_label =  data_loader(train_d)
# val_input, val_label =  data_loader(val_d)

In [0]:
# plk.dump(train_input, open(path + '/train_input.p', 'wb'))
# plk.dump(train_label, open(path + '/train_label.p', 'wb'))
# plk.dump(val_input, open(path+ '/val_input.p', 'wb'))
# plk.dump(val_label, open(path +'/val_label.p', 'wb'))

In [0]:
def save_file(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

out_filename = 'Balance sheet.txt'
save_file(all_data, out_filename)

In [0]:
# load doc into memory
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text


In [0]:
train_input = plk.load(open(path + '/train_input.p', 'rb'))
train_label = plk.load(open(path + '/train_label.p', 'rb'))
val_input = plk.load(open(path + '/val_input.p', 'rb'))
val_label = plk.load(open(path + '/val_label.p', 'rb'))

In [45]:
train_label.shape[1]

300

In [0]:
# sequences = tokenizer.texts_to_sequences(lines)
# vocab_size = len(tokenizer.word_index) + 1
# MAX_NUM_WORDS = words_to_load
# tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
# tokenizer.fit_on_texts(lines)
# sequences = tokenizer.texts_to_sequences(lines)
# word_index = tokenizer.word_index
# data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# X, y = data[:,:-1], data[:,-1]
vocab_size = loaded_embeddings_ft_en.shape[0]
# y = to_categorical(y, num_classes=vocab_size)
# seq_length = X.shape[1]
 
# define model
model = Sequential()
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[loaded_embeddings_ft_en],
                            input_length=MAX_SEQUENCE_LENGTH-1,
                            trainable=True)
model.add(embedding_layer)
model.add(Bidirectional(GRU(100, return_sequences=True)))
model.add(Bidirectional(GRU(100)))
model.add(Dense(100, activation='tanh'))
model.add(Dense(train_label.shape[1], activation='softmax'))

#import the checkpoint to save current model
filepath=path+"/GRU_1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
callbacks_list = [checkpoint, earlystopper]
# compile model

# rms = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# fit the model
model.fit(train_input, train_label, validation_data =(val_input, val_label), batch_size= 200, epochs=100, callbacks=callbacks_list)

 
# save the model to file
model.save(path+'/model_all.h5')
# save the tokenizer
plk.dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 300)           15193200  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 29, 200)           240600    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               180600    
_________________________________________________________________
dense_3 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_4 (Dense)              (None, 300)               30300     
Total params: 15,664,800
Trainable params: 15,664,800
Non-trainable params: 0
_________________________________________________________________
Train on 129327 samples, validate on 39485 samples
Epoch 1/100
129327/129327 [==============================] - 195s 2ms/step - lo

In [0]:
from random import randint
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
# generate a sequence from a language model
def generate_seq(model, seed_text_idx, n_words):
    bleu_score = []
    result = list()
    target_text = lines[seed_text_idx]
    seed_text = ' '.join(target_text.split()[:10])
    target_text_test = ' '.join(target_text.split()[10:])
    in_text = seed_text
    
    for _ in range(n_words):
    # encode the text as integer
#         encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = [words_ft_en[x] if x in ordered_words_ft_en else UNK_IDX for x in in_text.split()]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=MAX_SEQUENCE_LENGTH-1, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = idx2words_ft_en[yhat[0]]
        if yhat[0] == EOS_IDX:
          break
    # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    seq = ' '.join(result)
    ret_seq = seed_text + ' '+seq
    bleu_score = sentence_bleu(seq, target_text_test)
    return ret_seq, target_text, bleu_score

In [91]:
#test 1
# load cleaned text sequences
in_filename = 'Balance sheet.txt'
doc = load_file(in_filename)
lines = ['<s> '+x+ ' </s>' for x in doc.split('\n') if x != '']
# seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model(path+'/GRU_1.hdf5')
 
# # load the tokenizer
# tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text_idx = randint(0,len(lines))
 
# generate new text
generated, target, bleu = generate_seq(model, seed_text_idx, 50)
print(target)
print(generated)
print('BLEU score is: ', bleu)

<s> I appreciate that. And maybe one last one. Just curious, the AOCI jumped from $12B to $17B. I know spreads widened out a lot, but rates came down. Just curious if you can give us a little color behind the scenes?   </s>
<s> I appreciate that. And maybe one last one. Just research research research research research make make make make make does does does does does does does does research research by by by by by by by by by by by by by by by by by by by by by by by by by by by by by by
BLEU score is:  0.02062403823169552
